In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import ants
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from src.data.utils import get_all_images, convert_to_np_array, read_file
%matplotlib inline
import glob
from tqdm import tqdm

/home/shubham/anaconda3/envs/brain3dgpu/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: No module named 'src'

In [4]:
path = '/home/shubham/Documents/in-meds-brain-segmentation/cerebellum_dataset/'
# IN_PATH = Path(path)
paths = glob.glob(path+'*')

In [5]:
filepaths_input = []
filepaths_output = []
train_test = []
outlabel = []
pixel_count = []
img_shapes = []

for i in tqdm(paths):
    if np.random.randint(10) <=8:
        train = True
    else:
        train = False
    file_id = str(i.rsplit('/', 1)[-1])
    mri_loc = path+file_id+'/'+file_id+'.nii.gz'
    cerebellum_loc = path+file_id+'/'+file_id+'_cerebellum.nii.gz'
    mri_img = ants.image_read(mri_loc)
    cerebellum_img = ants.image_read(cerebellum_loc)
    mri_np = mri_img.numpy()
    cerebellum_np = cerebellum_img.numpy()
    slices = mri_np.shape[2]
    for sli in range(slices):
        mri_np_slice = mri_np[:, :, sli]
        cerebellum_np_slice = cerebellum_np[:, :, sli]
        save_path_mri = '/home/shubham/Documents/in-meds-brain-segmentation/mris/{}_{}'.format(file_id, sli)
        save_path_cerebellum = '/home/shubham/Documents/in-meds-brain-segmentation/cerebellum/{}_{}'.format(file_id, sli)
        np.save(save_path_mri, mri_np_slice)
        np.save(save_path_cerebellum, cerebellum_np_slice)
        train_test.append(train)
        filepaths_input.append(save_path_mri+".npy")
        filepaths_output.append(save_path_cerebellum+".npy")
        left_cerebellum = cerebellum_np_slice[cerebellum_np_slice==2].shape[0]
        right_cerebellum = cerebellum_np_slice[cerebellum_np_slice==1].shape[0]
        pixel_count.append((left_cerebellum, right_cerebellum))
        if np.sum(cerebellum_np_slice) > 0:
            outlabel.append(1)
        else:
            outlabel.append(0)
        h,w = mri_np_slice.shape[:2]
        img_shapes.append((h, w))

100%|██████████| 21/21 [00:06<00:00,  3.11it/s]


In [6]:
data = pd.DataFrame([filepaths_input, filepaths_output, train_test, outlabel, pixel_count, img_shapes]).T
data.columns = ["img_path", "label_path", "train_test", "cerebellum_present_or_not", "pixel_count", "img_shapes"]
print(data.shape)
data.head()

(3472, 6)


,img_path,label_path,train_test,cerebellum_present_or_not,pixel_count,img_shapes
0,/home/shubham/Documents/in-meds-brain-segmenta...,/home/shubham/Documents/in-meds-brain-segmenta...,True,0,"(0, 0)","(256, 256)"
1,/home/shubham/Documents/in-meds-brain-segmenta...,/home/shubham/Documents/in-meds-brain-segmenta...,True,0,"(0, 0)","(256, 256)"
2,/home/shubham/Documents/in-meds-brain-segmenta...,/home/shubham/Documents/in-meds-brain-segmenta...,True,0,"(0, 0)","(256, 256)"
3,/home/shubham/Documents/in-meds-brain-segmenta...,/home/shubham/Documents/in-meds-brain-segmenta...,True,0,"(0, 0)","(256, 256)"
4,/home/shubham/Documents/in-meds-brain-segmenta...,/home/shubham/Documents/in-meds-brain-segmenta...,True,0,"(0, 0)","(256, 256)"


In [7]:
data.to_csv("/home/shubham/Documents/in-meds-brain-segmentation/pytorch/segmentation_cerebellum.csv", index=False)

In [1]:
import src

ModuleNotFoundError: No module named 'src'